In [6]:
## importar paquetes al realizar modificaciones
%reload_ext autoreload
%autoreload 2

In [1]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

current_date=Timestamp('2023-04-28 14:00:00')


In [2]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/30877
Connected. Call `.close()` to terminate connection gracefully.
Fetching data from 2023-03-31 14:00:00 to 2023-04-28 13:00:00


2023-04-28 09:36:07,506 INFO: USE `taxi_demand_project_featurestore`
2023-04-28 09:36:08,277 INFO: SELECT `fg0`.`pickup_hour` `pickup_hour`, `fg0`.`rides` `rides`, `fg0`.`pickup_location_id` `pickup_location_id`
FROM `taxi_demand_project_featurestore`.`time_series_hourly_feature_group_1` `fg0`
WHERE `fg0`.`pickup_hour` >= TIMESTAMP '2023-03-30 02:00:00.000' AND `fg0`.`pickup_hour` <= TIMESTAMP '2023-04-29 01:00:00.000'


ts_data=               pickup_hour  rides  pickup_location_id
162448 2023-03-31 14:00:00      4                   1
2411   2023-03-31 15:00:00      4                   1
77652  2023-03-31 16:00:00      0                   1
38208  2023-03-31 17:00:00      6                   1
37478  2023-03-31 18:00:00      4                   1
...                    ...    ...                 ...
139597 2023-04-28 09:00:00     26                 265
139559 2023-04-28 10:00:00     26                 265
140359 2023-04-28 11:00:00     18                 265
139055 2023-04-28 12:00:00     22                 265
139891 2023-04-28 13:00:00     64                 265

[172704 rows x 3 columns]


In [3]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model,features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/30877
Connected. Call `.close()` to terminate connection gracefully.

In [4]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,227,0.0,2023-04-28 14:00:00
1,59,0.0,2023-04-28 14:00:00
2,192,0.0,2023-04-28 14:00:00
3,156,0.0,2023-04-28 14:00:00
4,189,1.0,2023-04-28 14:00:00
...,...,...,...
252,134,1.0,2023-04-28 14:00:00
253,159,1.0,2023-04-28 14:00:00
254,90,245.0,2023-04-28 14:00:00
255,214,0.0,2023-04-28 14:00:00


In [8]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id','pickup_hour'],
    event_time ='pickup_hour'
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/30877
Connected. Call `.close()` to terminate connection gracefully.


In [9]:
feature_group.insert(predictions,write_options={"wait_for_job":False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/30877/fs/30797/fg/40741


Uploading Dataframe: 0.00% |          | Rows 0/257 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/30877/jobs/named/model_predictions_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x1a9245b2670>, None)